Itererer gjennom listen med datapunkter og oppretter trajectories.

Det opprettes en trajectory, der alle påfølgende punkter legges inn, så lenge det er samme sjåfør, innenfor et tidsvindu og innenfor et bounding rectangle.

Et trajectory blir lagret dersom det er minst MIN_LEN punkter langt, er innenfor MIN_DIST og MAX_DIST, samt at avstanden mellom start og sluttpunkt ikke er mer enn 2.5 ganger sporets reelle lengde.

Sporene blir konstruert slik at siste punkt i et spor, kan ha sitt neste (naturlige) punkt i det neste genererte sporet, såfremt det tilfreddsstiller de andre kravene under generering. Dette bør ikke bli et problem, men er greit å opplyse om i oppgaven. De genererte sporene fra Roma-settet vil trolig ha mer komplisert data enn Porto, da det ikke er gitt om de genererte sporene er faktiske taxiturer, deler av taxiturer, snirkling - det kan være hva som helst, og mest sannsynlig en blanding av flere kategorier av kjøretyper.

In [18]:
# Importing necessary modules
import pandas as pd
# from tqdm import tqdm
import math
import re
import numpy as np

from datetime import datetime, timedelta
import os, shutil
from haversine import haversine, Unit

def calculate_trajectory_distance(positions: list[tuple[float]]) -> float:
    """
    Calculate the trajectory distance for a trajectory

    :param: List of coordinates (lat, lon)
    
    :return: Float (km) -> Combined distance between all pairs of points in km
    """
    distance = 0
    for i in range(1, len(positions)):
        from_location = positions[i-1]
        to_location = positions[i]

        distance += haversine(from_location, to_location, unit=Unit.KILOMETERS)
    return distance



In [19]:
# Declaring variables and constants for this sheet

SHOULD_DELETE_OLD_FILES= True

OUTPUT_FOLDER = "../rome/output/"
RAW_DATA_FILE = "../dataset/rome/rome_raw.txt"
CLEAN_DATA_FILE = "../dataset/rome/rome_clean.csv"


LOG = False # Set to true for printing/debug during data extraction

MAX_LON = 12.53
MIN_LON = 12.44
MAX_LAT = 41.93
MIN_LAT = 41.88

MIN_LEN = 40 # Perhaps change this one
MIN_DIST = 4 # In km
MAX_DIST = 6 # In km

MAX_TIME_DIFF_SECONDS = 32 # Maximum time difference between two consecutive trackpoints in a trajectory
MAX_DIST_BETWEEN_LOCATIONS = 0.4 # Km

NUMBER_OF_TRACES = 10

X = calculate_trajectory_distance([ ( MIN_LAT, MIN_LON ) , ( MAX_LAT, MIN_LON ) ])
Y = calculate_trajectory_distance([ ( MIN_LAT, MIN_LON ) , ( MIN_LAT, MAX_LON ) ])

In [20]:
# Reading the dataset into a dataframe

raw_df = pd.read_csv(RAW_DATA_FILE, delimiter=";", names=["index", "timestamp", "location"])

print(raw_df.head(10))

   index                      timestamp  \
0    156  2014-02-01 00:00:00.739166+01   
1    187  2014-02-01 00:00:01.148457+01   
2    297  2014-02-01 00:00:01.220066+01   
3     89  2014-02-01 00:00:01.470854+01   
4     79  2014-02-01 00:00:01.631136+01   
5    191  2014-02-01 00:00:02.048546+01   
6    343  2014-02-01 00:00:02.647839+01   
7    341  2014-02-01 00:00:02.709888+01   
8    260  2014-02-01 00:00:03.458195+01   
9     59  2014-02-01 00:00:03.707117+01   

                                   location  
0  POINT(41.8836718276551 12.4877775603346)  
1  POINT(41.9285433333333 12.4690366666667)  
2  POINT(41.8910686119733 12.4927045625339)  
3  POINT(41.7931766914244 12.4321219603157)  
4            POINT(41.90027472 12.46274618)  
5  POINT(41.8523047579646 12.5774065771898)  
6  POINT(41.8921718255185 12.4696996165151)  
7  POINT(41.9102125627332 12.4770004336041)  
8  POINT(41.8658208551143 12.4655221109313)  
9  POINT(41.8967831636848 12.4821987021152)  


In [21]:
# Must sort the values according to their index and timestamp 
# Also copying the dataframe as we will touch the values in the "location" column
# Will probably take some time

df = raw_df.head(1500000).sort_values(by=["index", "timestamp"]).copy()

# Deleting raw_df for performance reasons
del raw_df


In [22]:
# Cell that translates the location column of the dataframe to only the locations
# Will probably take some time to finish

df["location"] = df["location"].apply(lambda x: re.search(r'\(.*?\)', x).group(0)[1:-1].replace(" ", ","))


In [23]:
# Run this cell to clear the chosen files in the PORTO folder

if SHOULD_DELETE_OLD_FILES:
    for filename in os.listdir(OUTPUT_FOLDER):
        file_path = os.path.join(OUTPUT_FOLDER, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print("Failed to remove %s. Reason: %s" % (file_path, e))

In [28]:
# Cell to extract traces that are of a minimum length of N locations and within a given distance window
counter = 0

last_driver_id = None
last_timestamp = None
last_location = None
trajectory = []

cleaned_trajectories = []
for index, row in df.iterrows():

    current_driver_id = row["index"]
    current_timestamp = row["timestamp"]
    
    current_location = row["location"].split(",")
    lat, lon = current_location

    # On first run or change in driver_id:
    if (not last_timestamp) or (last_driver_id != current_driver_id):
        last_timestamp = current_timestamp
        last_location = current_location
        last_driver_id = current_driver_id
        trajectory.clear()
        continue

    # Location outside bounded rectangle, go to next row
    if ( not ( MIN_LON <= float(lon) <= MAX_LON )) or ( not ( MIN_LAT <= float(lat) <= MAX_LAT )):
        if LOG: print("Location outside bounded rectangle: ", lat,lon)
        continue
    
    try:

        # Checking whether time between two consecutive points are greater than a given const 
        t_last = datetime.strptime(last_timestamp[0:last_timestamp.index(".")], "%Y-%m-%d %H:%M:%S")
        t_current = datetime.strptime(current_timestamp[0:current_timestamp.index(".")], "%Y-%m-%d %H:%M:%S")

    except:
        last_timestamp = current_timestamp
        last_location = current_location
        last_driver_id = current_driver_id
        trajectory.clear()
        continue


    # If timedelta small enough
    if t_current - t_last <= timedelta(seconds=MAX_TIME_DIFF_SECONDS):
        if LOG: print("Timedelta is small enough")
        
        trajectory.append((float(lat), float(lon)))

    if MIN_LEN <= len(trajectory):
        trajectory_distance = calculate_trajectory_distance(trajectory)
        shortest_distance = calculate_trajectory_distance([trajectory[0],trajectory[-1]])

        if  MIN_DIST <= trajectory_distance <= MAX_DIST and trajectory_distance < shortest_distance * 2.5:
            # TODO Ensure that all points in trajectory are valid
            
            old_loc = trajectory[0]
            for loc in trajectory:
                if old_loc != loc:
                    distance = calculate_trajectory_distance([(old_loc), (loc)])

                    # If distance between two consecutive coordinates are too great:
                    if distance > MAX_DIST_BETWEEN_LOCATIONS:
                        if LOG: print("Possibly noisy data - continuing with next trace")
                        break 

                old_loc = loc
            
            else:            
            
                # Saving trajectory and updating variables
                # print("------------------------------------")
                # print(trajectory)
                # coordinates_lists = [[lat, lon] for lat, lon in trajectory]
                cleaned_trajectories.append([current_driver_id, current_timestamp, [float(lat), float(lon)]])
                if LOG: print(f"Total distance: {trajectory_distance, shortest_distance, len(trajectory)}")

                counter +=1
            
            trajectory.clear()

        if trajectory_distance > MAX_DIST:
            trajectory.clear()

    
    # If enough trajetories are generated: 
    # if counter >= NUMBER_OF_TRACES: 
    #     break
    
    last_timestamp = current_timestamp
    # last_driver_id = last_driver_id
    # last_location = last_location
print(cleaned_trajectories[0:10])
cleaned_df = pd.DataFrame(cleaned_trajectories, columns=["index", "timestamp", "location"])
cleaned_df.to_csv(CLEAN_DATA_FILE, index=False)

print(f"Created {counter} trajectories")

[[2, '2014-02-01 00:14:10.132878+01', [41.9033913036255, 12.4876087503095]], [2, '2014-02-01 01:14:53.440003+01', [41.8999142436576, 12.4441906822934]], [2, '2014-02-01 01:52:19.7449+01', [41.8959594244013, 12.4496450994481]], [2, '2014-02-01 02:58:57.352827+01', [41.8824816421777, 12.454844364531]], [2, '2014-02-01 04:42:58.23288+01', [41.9077406913423, 12.4910772764437]], [2, '2014-02-01 21:42:43.685802+01', [41.9053566960219, 12.4493586509094]], [2, '2014-02-01 21:58:53.645706+01', [41.9031474155762, 12.4867018941079]], [2, '2014-02-01 23:40:25.264024+01', [41.9127874299986, 12.4929061846492]], [2, '2014-02-02 04:58:28.729974+01', [41.9126299945405, 12.4845480461226]], [2, '2014-02-02 05:19:42.090741+01', [41.9129688470329, 12.4502509292558]]]
Created 3059 trajectories
